# Hierarchical Reasoning Model (Layers 0-5)

This notebook demonstrates a conceptual implementation of a Hierarchical Reasoning Model (HRM). The model is structured into six distinct layers, from token-level processing up to meta-reasoning. Each layer builds upon the output of the previous one, creating a rich, multi-faceted analysis of the input text.

## 1. Setup and Imports

First, we import the necessary libraries. `dotenv` is used for loading environment variables, `os` for interacting with the operating system, `spacy` for linguistic features, and `openai` for interacting with the OpenAI API.

In [12]:
from dotenv import load_dotenv
import os
import spacy
from openai import OpenAI

### Environment Configuration

This section loads environment variables from a `.env` file. This is a secure way to manage sensitive information like API keys.

**Note:** The cell below that changes the directory (`os.chdir`) contains a hardcoded path. This is for demonstration purposes and should be adapted for your own environment if you wish to run it.

In [7]:
direct = r"C:\Users\ajean\Downloads"
os.chdir(direct)

In [17]:
load_dotenv()  # This loads variables from .env into os.environ

endpoint = os.environ["OPENAI_EMBED_END_POINT"]
deployment = os.environ["OPENAI_EMBED_DEPLOYMENT"]
api_key = os.environ["OPENAI_EMBED_API_KEY"]
chat_model = os.environ["OPENAI_CHAT_DEPLOYMENT"]

## 2. Defining the HRM Layers

Here we define the classes for each of the 6 layers in our hierarchical model.

### Layer 0: Token Embedding Layer

This is the foundational layer. It takes raw text as input and converts it into a numerical representation (embedding) using a pre-trained model from the OpenAI API. This embedding captures the semantic meaning of the text.

In [31]:
# ---------- Layer 0: Token Embeddings via OpenAI Embedding Endpoint ----------
class TokenEmbeddingLayer:
    def __init__(self):
        endpoint = os.environ["OPENAI_EMBED_END_POINT"] #
        deployment = "text-embedding-3-large"
        api_key = os.environ["OPENAI_EMBED_API_KEY"] #
        self.client = OpenAI(
            base_url=endpoint.rstrip("/") + "/openai/v1/",
            api_key=api_key
        )
        self.deployment = deployment

    def forward(self, text):
        response = self.client.embeddings.create(
            input=text,
            model=self.deployment
        )
        return {"embedding": response.data[0].embedding}

### Layer 1: Syntax Layer

The Syntax Layer processes the text to understand its grammatical structure. It uses the `spaCy` library to perform tasks like Part-of-Speech (POS) tagging and dependency parsing.

In [32]:
# ---------- Layer 1: Syntax ----------
class SyntaxLayer:
    def __init__(self, spacy_model="en_core_web_sm"):
        self.nlp = spacy.load(spacy_model)

    def forward(self, text):
        doc = self.nlp(text)
        syntax_info = [(tok.text, tok.pos_, tok.dep_, tok.head.text) for tok in doc]
        return syntax_info

### Layer 2: Semantic Layer

This layer focuses on the meaning of the text. It performs Named Entity Recognition (NER) to identify real-world objects like people and places, and it attempts to extract basic relationships between them.

In [ ]:
# ---------- Layer 2: Semantics ----------
class SemanticLayer:
    def __init__(self, spacy_model="en_core_web_sm"):
        self.nlp = spacy.load(spacy_model)

    def forward(self, text):
        doc = self.nlp(text)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        relations = []
        for tok in doc:
            if tok.dep_ in ("ROOT", "relcl") and tok.pos_ == "VERB":
                subj = [w.text for w in tok.lefts if w.dep_.startswith("nsubj")]
                obj = [w.text for w in tok.rights if w.dep_.startswith("dobj")]
                if subj and obj:
                    relations.append((subj[0], tok.lemma_, obj[0]))
        return {"entities": entities, "relations": relations}

### Layer 3: Discourse Layer

The Discourse Layer analyzes how sentences and clauses are connected. It identifies discourse markers (e.g., 'because', 'however') and uses an OpenAI Chat model to perform coreference resolution (identifying when different words refer to the same entity, like 'John' and 'he').

In [ ]:
# ---------- Layer 3: Discourse + Coreference via OpenAI Chat API ----------
class DiscourseLayer:
    def __init__(self):
        endpoint = os.environ["OPENAI_EMBED_END_POINT"] # 
        api_key = os.environ["OPENAI_EMBED_API_KEY"] # 
        chat_model = "o4-mini" # Replace with your actual chat deployment name
        self.client = OpenAI(
            base_url=endpoint.rstrip("/") + "/openai/v1/",
            api_key=api_key
        )
        self.model = chat_model
        self.discourse_markers = [
            "however", "therefore", "because", "although", "furthermore",
            "meanwhile", "then", "afterward", "but", "so"
        ]

    def coreference_prompt(self, text):
        return (
            f"Text: {text}\n"
            "Task: List all coreference clusters. For each cluster, show all mentions and explain what entity they refer to."
        )

    def forward(self, text):
        tokens = text.split()
        discourse_signals = [tok for tok in tokens if tok.lower() in self.discourse_markers]
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": self.coreference_prompt(text)}]
        )
        coref_result = response.choices[0].message.content
        return {
            "coreferences": coref_result,
            "discourse_markers": discourse_signals
        }

### Layer 4: Pragmatic Layer

This layer deals with the context-dependent meaning of the text. It uses an OpenAI Chat model to identify the speaker's intent and the real-world knowledge required to fully understand the text.

In [ ]:
# ---------- Layer 4: Pragmatics (Intent + World Knowledge via OpenAI Chat API) ----------
class PragmaticLayer:
    def __init__(self):
        endpoint =  os.environ["OPENAI_EMBED_END_POINT"] #  
        api_key = os.environ["OPENAI_EMBED_API_KEY"] # 
        chat_model = "o4-mini"
        self.client = OpenAI(
            base_url=endpoint.rstrip("/") + "/openai/v1/",
            api_key=api_key
        )
        self.model = chat_model

    def pragmatics_prompt(self, text):
        return (
            f"Text: {text}\n"
            "Task:\n"
            "1. Identify the main intent behind the text (the speaker’s goal, request, question, or communicative purpose).\n"
            "2. Summarize any background/world knowledge needed to fully understand or respond to the intent (include factual knowledge, context, norms, or relevant history if useful)."
        )

    def forward(self, text):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": self.pragmatics_prompt(text)}]
        )
        pragmatic_result = response.choices[0].message.content
        return {
            "pragmatic_analysis": pragmatic_result
        }

### Layer 5: Meta-Reasoning Layer

This is the highest level of reasoning. It takes the outputs from all previous layers and performs a meta-analysis. This includes decomposing the overall goal of the text, generating a plan, and checking for logical consistency across the different layers of analysis.

In [ ]:
# ---------- Layer 5: Meta-Reasoning (Goal decomposition & planning) ----------
class MetaReasoningLayer:
    def __init__(self):
        endpoint = os.environ["OPENAI_EMBED_END_POINT"] # 
        api_key = os.environ["OPENAI_EMBED_API_KEY"] #
        chat_model = "o4-mini"
        self.client = OpenAI(
            base_url=endpoint.rstrip("/") + "/openai/v1/",
            api_key=api_key
        )
        self.model = chat_model

    def metareasoning_prompt(self, text, outputs):
        """Prompt decomposes goals, plans steps, checks consistency across layers."""
        return (
            f"Original Text: {text}\n"
            f"Prior layer outputs:\n{outputs}\n"
            "Task:\n"
            "1. Decompose the overall goal of this passage into explicit subgoals.\n"
            "2. Generate a reasoning or action plan—step by step.\n"
            "3. Analyze outputs above for logical consistency, completeness, or errors. Highlight any potential issues or corrections."
        )

    def forward(self, text, prev_outputs):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": self.metareasoning_prompt(text, prev_outputs)}]
        )
        meta_result = response.choices[0].message.content
        return {
            "goal_decomposition": meta_result
        }

### Reasoning Tracker

To maintain transparency, the `ReasoningTracker` class logs the inputs, outputs, and reasoning type at each step of the process. This allows us to create an auditable trace of the model's 'thought process'.

In [ ]:
# ---------- Reasoning Tracker ----------
class ReasoningTracker:
    def __init__(self):
        self.chain = []

    def log(self, layer, input_repr, output_repr, reason_type):
        self.chain.append({
            "layer": layer,
            "input": str(input_repr),
            "output": str(output_repr),
            "reason_type": reason_type
        })

    def explain(self):
        explanation = []
        for step in self.chain:
            explanation.append(
                f"[{step['layer']}] {step['reason_type']}: input={step['input']} → output={step['output']}"
            )
        return "\n".join(explanation)

## 3. HRM Orchestrator and Demonstration

The `HRM_L0toL5` class ties all the layers together into a single pipeline. The `process` method executes the forward pass through each layer sequentially.

The `if __name__ == "__main__":` block provides a simple demonstration using a sample passage.

In [33]:
# ---------- Unified Hierarchical Modular Flow ----------
class HRM_L0toL5:
    def __init__(self):
        self.layer0 = TokenEmbeddingLayer()
        self.layer1 = SyntaxLayer()
        self.layer2 = SemanticLayer()
        self.layer3 = DiscourseLayer()
        self.layer4 = PragmaticLayer()
        self.layer5 = MetaReasoningLayer()
        self.tracker = ReasoningTracker()

    def process(self, text):
        tokens = self.layer0.forward(text)
        self.tracker.log("Layer 0: Tokens", text, "[embedding omitted]", "embedding generated")

        syntax = self.layer1.forward(text)
        self.tracker.log("Layer 1: Syntax", text, syntax, "parse dependencies")

        semantics = self.layer2.forward(text)
        self.tracker.log("Layer 2: Semantics", text, semantics, "extract entities/relations")

        discourse = self.layer3.forward(text)
        self.tracker.log("Layer 3: Discourse", text, discourse, "coreference")

        pragmatics = self.layer4.forward(text)
        self.tracker.log("Layer 4: Pragmatics", text, pragmatics, "intent & world knowledge")

        # For Layer 5, include all previous outputs for global planning and consistency checks
        prev_outputs = {
            "syntax": syntax,
            "semantics": semantics,
            "discourse": discourse,
            "pragmatics": pragmatics
        }
        meta = self.layer5.forward(text, prev_outputs)
        self.tracker.log("Layer 5: Meta-Reasoning", text, meta, "goal decomposition, planning, consistency check")

        return {
            "syntax": syntax,
            "semantics": semantics,
            "discourse": discourse,
            "pragmatics": pragmatics,
            "meta_reasoning": meta
        }

# ---------- DEMO ----------
if __name__ == "__main__":
    model = HRM_L0toL5()
    passage = "John bought a new car in New York. He drove it home because he was excited."
    final_output = model.process(passage)
    print("=== Hierarchical Output Including Meta-Reasoning ===")
    print(final_output)
    print("\n=== Reasoning Trace ===")
    print(model.tracker.explain())

=== Hierarchical Output Including Meta-Reasoning ===
{'syntax': [('John', 'PROPN', 'nsubj', 'bought'), ('bought', 'VERB', 'ROOT', 'bought'), ('a', 'DET', 'det', 'car'), ('new', 'ADJ', 'amod', 'car'), ('car', 'NOUN', 'dobj', 'bought'), ('in', 'ADP', 'prep', 'bought'), ('New', 'PROPN', 'compound', 'York'), ('York', 'PROPN', 'pobj', 'in'), ('.', 'PUNCT', 'punct', 'bought'), ('He', 'PRON', 'nsubj', 'drove'), ('drove', 'VERB', 'ROOT', 'drove'), ('it', 'PRON', 'dobj', 'drove'), ('home', 'ADV', 'advmod', 'drove'), ('because', 'SCONJ', 'mark', 'was'), ('he', 'PRON', 'nsubj', 'was'), ('was', 'AUX', 'advcl', 'drove'), ('excited', 'ADJ', 'acomp', 'was'), ('.', 'PUNCT', 'punct', 'drove')], 'semantics': {'entities': [('John', 'PERSON'), ('New York', 'GPE')], 'relations': [('John', 'buy', 'car'), ('He', 'drive', 'it')]}, 'discourse': {'coreferences': 'Here are the coreference clusters in the text:\n\n1. Cluster 1 (the person)\n   – Mentions:\n     • “John” (sentence 1 subject)  \n     • “He” (senten